# Import Libraries

In [1]:
import re
from typing import Dict, Any

# Define Functions

In [2]:
def query_router(user_query: str) -> Dict[str, Any]:
    """
    Routes user queries to either the recommender system or semantic search
    based on keyword detection.
    
    Args:
        user_query (str): The user's input query
        
    Returns:
        Dict containing:
            - route: str ("recommender" or "semantic_search")
            - query: original query string
            - confidence: routing confidence score (0.0-1.0)
            - matched_keywords: list of keywords that triggered the route
    """
    
    # Normalize query for analysis
    normalized_query = user_query.lower().strip()
    
    # Define recommender trigger patterns
    recommender_keywords = [
        # Direct recommendation requests
        "recommend", "suggest", "advise", "propose",
        # Variations and related terms
        "find me", "show me", "give me", "i want", "i need",
        "looking for", "search for", "help me find",
        # Portfolio-specific terms
        "portfolio", "investment", "stocks for", "best stocks",
        "top stocks", "good stocks", "suitable stocks",
        # Comparative requests
        "better than", "alternatives to", "similar to",
        # Action-oriented
        "should i", "what to", "which stocks"
    ]
    
    # Define patterns for more complex matching
    recommender_patterns = [
        r"\b(recommend|suggest|advise|propose)\b",
        r"\b(find|show|give)\s+me\b",
        r"\bi\s+(want|need|looking)\b",
        r"\b(best|top|good)\s+stocks?\b",
        r"\bstocks?\s+for\b",
        r"\bportfolio\b",
        r"\bshould\s+i\b",
        r"\bwhat\s+to\b",
        r"\bwhich\s+stocks?\b"
    ]
    
    matched_keywords = []
    pattern_matches = []
    
    # Check for keyword matches
    for keyword in recommender_keywords:
        if keyword in normalized_query:
            matched_keywords.append(keyword)
    
    # Check for pattern matches
    for pattern in recommender_patterns:
        if re.search(pattern, normalized_query):
            pattern_matches.append(pattern)
    
    # Calculate confidence based on matches
    total_matches = len(matched_keywords) + len(pattern_matches)
    
    if total_matches > 0:
        # Route to recommender
        route = "recommender"
    else:
        # Route to semantic search
        route = "semantic_search"
    
    # Check for specific factual queries that should go to semantic search
    factual_patterns = [
        r"what\s+is\s+the\s+\w+\s+of\s+[A-Z]+",  # "what is the price of AAPL"
        r"[A-Z]{1,5}\s+(price|return|yield|ratio)",  # "AAPL price"
        r"show\s+me\s+[A-Z]{1,5}",  # "show me AAPL" (specific ticker)
    ]
    
    for pattern in factual_patterns:
        if re.search(pattern, user_query):
            route = "semantic_search"
            matched_keywords = ["factual_query_pattern"]
            break
    
    return {
        "route": route,
        "query": user_query,
        "matched_keywords": matched_keywords,
        "pattern_matches": pattern_matches
    }

In [3]:
test_queries = [
    "Recommend me 3 stocks that have a high yield, low risk and have an annualized return of at least 20%",
    "What is the annualized return of AAPL?",
    "Find me the best tech stocks for my portfolio",
    "AAPL price and dividend yield",
    "I want stocks with low volatility",
    "Show me TSLA financial metrics",
    "What are good defensive stocks?",
    "Suggest some growth stocks",
    "Microsoft earnings report"
]

for query in test_queries:
    result = query_router(query)
    print(f"Query: {query}")
    print(f"Route: {result['route']}")
    print(f"Matched: {result['matched_keywords']}")
    print("-" * 50)

Query: Recommend me 3 stocks that have a high yield, low risk and have an annualized return of at least 20%
Route: recommender
Matched: ['recommend']
--------------------------------------------------
Query: What is the annualized return of AAPL?
Route: semantic_search
Matched: []
--------------------------------------------------
Query: Find me the best tech stocks for my portfolio
Route: recommender
Matched: ['find me', 'portfolio', 'stocks for']
--------------------------------------------------
Query: AAPL price and dividend yield
Route: semantic_search
Matched: ['factual_query_pattern']
--------------------------------------------------
Query: I want stocks with low volatility
Route: recommender
Matched: ['i want']
--------------------------------------------------
Query: Show me TSLA financial metrics
Route: recommender
Matched: ['show me']
--------------------------------------------------
Query: What are good defensive stocks?
Route: semantic_search
Matched: []
----------------